# Experiment 2: Code Solutions

## Where we're at
Up to this point we've coded up some basic infrastructure that makes it easy to run ARC prize items, where out business logic including Reflexion and giving the agent plenty of time to think fits into about a page.

We've run the first 20 problems in the training set and got 3/20 = 15%.  This is showing some non-zero signs of life and is a good baseline to start improving.

## Where we go from here
Looking at the sessions where it failed, it looks like it understands what to do but is having trouble with the details of the execution.  For example, on one of the items involving a flood fill, the agent understands what it needs to do and mostly does it, but has trouble geting every single cell right.

## Hypothesis: Python DSL
Can we do a thing where it writes code to do it?  Others on the leaderboard have done this using their own DSL and fine tuning.  Maybe we can make a collection of Python functions with libraries that make a lot of these tasks as easy as practical.  This way we leverage the model's deep understanding of Python.

## Next steps:
* [ ] **Verify**: Does it actually understand the patterns? Review the cases where it failed.  If it can code strong, will that actually help? It won't help if the agent did not actually understand the pattern.  My hypothesis is grounded in the assumption that mostly it does understand what it needs to do, but it is struggling with the **execution** of what it needs to do.  Verify this assumption.
* [ ] **Prompt**: Put together a prompt including what you propose doing, and see how the LLM does with that.  This way I can verify the promise of this approach before I actually build all of it. This is a common trick I use with LLM projects: verify the promise of the approach before I build the infrastructure needed to make it happen.
* [ ] **Infrastructure:**  Assuming the above two things turn out like I expect, some ideas of what to do with it are:
    * **Disjoint sets**.  Make a great abstraction for working with disjoint sets, as I have done in the past for working with datacenter tiles.
    * **Coding agent.**  Make a coding agent
    * **Module describer.** Make a thing that describes all the public classes and methods of a module, so that I'm spending more time writing code and less time writng prompts.